# "Understanding The Data -- Kovaaks Aim Trainer"
> "A Deep Dive into the automatically saved csvs from Kovaaks Aim Trainer"

- toc: true
- branch: master
- badges: true
- comments: true
- author: Josh Prier
- categories: [Kovaaks, Understanding The Data, Data Science]

# Kovaaks
Kovaaks is a "game" that allows to directly practice mouse control in 3d fps games. There are hundreds of different mini-games to practice with, each having a different focus. 

This [guide](https://www.dropbox.com/s/vaba3potfhf9jy1/KovaaK%20aim%20workout%20routines.pdf?dl=0) is best for getting perspective or understanding how and why to use Kovaaks


# The Data
Kovaaks has a great feature in which it saves every mini-game's stats to a csv. 
The format of the csv's names is as follows
```
<scenario name> - <Challenge or Freeplay> - YYYY.MM.DD-HH.MM.SS Stats.csv
```
Example:
```
Tile Frenzy - Challenge - 2020.12.14-08.46.00 Stats.csv
```